In [2]:
import os
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
load_dotenv()

True

In [3]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),

)



In [4]:
llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            temperature=0.1,
            max_tokens=1000
        )

C:\Users\Rohan Gupta\AppData\Local\Temp\ipykernel_46340\3409123018.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [4]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_transformer = LLMGraphTransformer(llm=llm )

In [7]:
import pandas as pd
df=pd.read_csv("data/Diseases_Symptoms.csv")
df.head()

,Code,Name,Symptoms,Treatments
0,1,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o...","Antidepressant medications, Cognitive Behavior..."
1,2,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue","Voice Rest, Speech Therapy, Surgical Removal"
2,3,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck...","Growth hormone therapy, Estrogen replacement t..."
3,4,Cryptorchidism,"Absence or undescended testicle(s), empty scro...",Observation and monitoring (in cases of mild o...
4,5,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala...","Supportive Measures, Gastric Decontamination, ..."


In [8]:
# Let's explore the data structure first
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

Dataset Info:
Shape: (400, 4)
Columns: ['Code', 'Name', 'Symptoms', 'Treatments']

First few rows:
   Code                         Name  \
0     1               Panic disorder   
1     2             Vocal cord polyp   
2     3              Turner syndrome   
3     4               Cryptorchidism   
4     5  Ethylene glycol poisoning-1   

                                            Symptoms  \
0  Palpitations, Sweating, Trembling, Shortness o...   
1           Hoarseness, Vocal Changes, Vocal Fatigue   
2  Short stature, Gonadal dysgenesis, Webbed neck...   
3  Absence or undescended testicle(s), empty scro...   
4  Nausea, vomiting, abdominal pain, General mala...   

                                          Treatments  
0  Antidepressant medications, Cognitive Behavior...  
1       Voice Rest, Speech Therapy, Surgical Removal  
2  Growth hormone therapy, Estrogen replacement t...  
3  Observation and monitoring (in cases of mild o...  
4  Supportive Measures, Gastric Decontamination,

In [9]:
# Step 1: Create structured documents for graph transformation
from langchain_core.documents import Document

def create_medical_documents(df):
    """Convert CSV rows into structured documents for graph transformation"""
    documents = []
    
    for _, row in df.iterrows():
        # Clean and prepare the data
        code = str(row['Code'])
        name = str(row['Name']).strip()
        symptoms = str(row['Symptoms']).strip() if pd.notna(row['Symptoms']) else ""
        treatments = str(row['Treatments']).strip() if pd.notna(row['Treatments']) else ""
        
        # Create a structured text that clearly defines relationships
        content = f"""
Medical Condition: {name}
Code: {code}

Symptoms:
{symptoms}

Treatments:
{treatments}

The medical condition {name} has the following symptoms: {symptoms}.
The medical condition {name} can be treated with: {treatments}.
"""
        
        # Create document with metadata
        doc = Document(
            page_content=content.strip(),
            metadata={
                'code': code,
                'condition_name': name,
                'type': 'medical_condition'
            }
        )
        documents.append(doc)
    
    return documents

# Create documents from our CSV data
print("Creating structured documents...")
medical_documents = create_medical_documents(df)
print(f"Created {len(medical_documents)} medical documents")

# Let's look at the first document to see the structure
print("\nExample document:")
print(medical_documents[0].page_content[:500] + "...")
print("\nMetadata:", medical_documents[0].metadata)

Creating structured documents...
Created 400 medical documents

Example document:
Medical Condition: Panic disorder
Code: 1

Symptoms:
Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness

Treatments:
Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques

The medical condition Panic disorder has the following symptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness.
The medical condition Panic disorder can be treated with: Antidepressant medications, Cognitive Behavioral ...

Metadata: {'code': '1', 'condition_name': 'Panic disorder', 'type': 'medical_condition'}


In [10]:
# Step 2: Transform documents to graph documents
print("Transforming documents to graph format...")
print("This may take a few minutes as we're processing 400 medical conditions...")

# Process documents in batches to avoid overwhelming the LLM
batch_size = 10  # Process 10 documents at a time
all_graph_documents = []

for i in range(0, len(medical_documents), batch_size):
    batch = medical_documents[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1} of {(len(medical_documents) + batch_size - 1)//batch_size} ({len(batch)} documents)...")
    
    try:
        # Transform batch to graph documents
        batch_graph_docs = llm_transformer.convert_to_graph_documents(batch)
        all_graph_documents.extend(batch_graph_docs)
        print(f"  ✓ Successfully processed batch {i//batch_size + 1}")
    except Exception as e:
        print(f"  ✗ Error processing batch {i//batch_size + 1}: {str(e)}")
        continue

print(f"\nCompleted! Generated {len(all_graph_documents)} graph documents")

# Let's examine the first graph document
if all_graph_documents:
    print(f"\nFirst graph document has:")
    print(f"- {len(all_graph_documents[0].nodes)} nodes")
    print(f"- {len(all_graph_documents[0].relationships)} relationships")
    
    # Show some example nodes
    print("\nExample nodes:")
    for i, node in enumerate(all_graph_documents[0].nodes[:5]):
        print(f"  {i+1}. {node.type}: '{node.id}'")
        
    # Show some example relationships
    print("\nExample relationships:")
    for i, rel in enumerate(all_graph_documents[0].relationships[:5]):
        print(f"  {i+1}. ({rel.source.id}) --[{rel.type}]--> ({rel.target.id})")
else:
    print("No graph documents were created. Please check for errors above.")

Transforming documents to graph format...
This may take a few minutes as we're processing 400 medical conditions...
Processing batch 1 of 40 (10 documents)...
  ✓ Successfully processed batch 1
Processing batch 2 of 40 (10 documents)...
  ✓ Successfully processed batch 2
Processing batch 3 of 40 (10 documents)...
  ✓ Successfully processed batch 3
Processing batch 4 of 40 (10 documents)...
  ✓ Successfully processed batch 4
Processing batch 5 of 40 (10 documents)...
  ✓ Successfully processed batch 5
Processing batch 6 of 40 (10 documents)...
  ✓ Successfully processed batch 6
Processing batch 7 of 40 (10 documents)...
  ✓ Successfully processed batch 7
Processing batch 8 of 40 (10 documents)...
  ✓ Successfully processed batch 8
Processing batch 9 of 40 (10 documents)...
  ✓ Successfully processed batch 9
Processing batch 10 of 40 (10 documents)...
  ✓ Successfully processed batch 10
Processing batch 11 of 40 (10 documents)...
  ✓ Successfully processed batch 11
Processing batch 12 of

In [11]:
# Step 3: Load graph documents into Neo4j
print("Loading graph documents into Neo4j database...")
print("This may take a few minutes...")

try:
    # Clear existing data if any (optional - remove this if you want to keep existing data)
    graph.query("MATCH (n) DETACH DELETE n")
    print("Cleared existing data from Neo4j")
    
    # Load all graph documents into Neo4j
    graph.add_graph_documents(all_graph_documents)
    print("✓ Successfully loaded all graph documents into Neo4j!")
    
except Exception as e:
    print(f"✗ Error loading data into Neo4j: {str(e)}")
    print("Please check your Neo4j connection and try again.")

Loading graph documents into Neo4j database...
This may take a few minutes...
Cleared existing data from Neo4j
✓ Successfully loaded all graph documents into Neo4j!


In [6]:
# Step 4: Verify the data import
print("Verifying data import in Neo4j...")

# Count total nodes
node_count_query = "MATCH (n) RETURN count(n) as total_nodes"
result = graph.query(node_count_query)
total_nodes = result[0]['total_nodes']
print(f"Total nodes created: {total_nodes}")

# Count total relationships
rel_count_query = "MATCH ()-[r]->() RETURN count(r) as total_relationships"
result = graph.query(rel_count_query)
total_relationships = result[0]['total_relationships']
print(f"Total relationships created: {total_relationships}")

# Show node types distribution
node_types_query = """
MATCH (n) 
RETURN labels(n)[0] as node_type, count(n) as count 
ORDER BY count DESC
"""
result = graph.query(node_types_query)
print(f"\nNode types distribution:")
for row in result:
    print(f"  {row['node_type']}: {row['count']}")

# Show relationship types distribution
rel_types_query = """
MATCH ()-[r]->() 
RETURN type(r) as relationship_type, count(r) as count 
ORDER BY count DESC
"""
result = graph.query(rel_types_query)
print(f"\nRelationship types distribution:")
for row in result:
    print(f"  {row['relationship_type']}: {row['count']}")

# Show some example data
example_query = """
MATCH (d:Disease)-[r]->(s)
RETURN d.id as disease, type(r) as relationship, s.id as connected_entity
LIMIT 10
"""
result = graph.query(example_query)
print(f"\nExample connections:")
for row in result:
    print(f"  {row['disease']} --[{row['relationship']}]--> {row['connected_entity']}")

print("\n✓ Data verification complete!")

Verifying data import in Neo4j...
Total nodes created: 2491
Total relationships created: 3358

Node types distribution:
  Treatment: 1128
  Symptom: 947
  Medical Condition: 392
  Medication: 12
  Procedure: 4
  Therapy: 4
  Therapy Type: 2
  Node: 1
  Cause: 1

Relationship types distribution:
  HAS_SYMPTOM: 1624
  CAN_BE_TREATED_WITH: 1104
  TREATED_WITH: 324
  CAN_BE_TREATED_BY: 164
  HAS_TREATMENT: 56
  TREATED_BY: 49
  INCLUDES: 32
  INVOLVES_PROCEDURE: 2
  USES_MEDICATION: 2
  HAS_CAUSE: 1


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Disease)} {position: line: 2, column: 10, offset: 10} for query: '\nMATCH (d:Disease)-[r]->(s)\nRETURN d.id as disease, type(r) as relationship, s.id as connected_entity\nLIMIT 10\n'



Example connections:

✓ Data verification complete!


In [13]:
# Show some example connections with correct node label
example_query = """
MATCH (d:`Medical Condition`)-[r]->(s)
RETURN d.id as disease, type(r) as relationship, labels(s)[0] as target_type, s.id as connected_entity
LIMIT 10
"""
result = graph.query(example_query)
print(f"Example medical condition connections:")
for row in result:
    print(f"  '{row['disease']}' --[{row['relationship']}]--> {row['target_type']}: '{row['connected_entity']}'")

print(f"\n🎉 Knowledge graph successfully created with:")
print(f"   • {total_nodes} nodes (392 medical conditions, 947 symptoms, 1128 treatments)")
print(f"   • {total_relationships} relationships")
print(f"   • Primary relationship types: HAS_SYMPTOM ({1624}), CAN_BE_TREATED_WITH ({1104})")

Example medical condition connections:
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Fear of losing control'
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Trembling'
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Palpitations'
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Shortness of breath'
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Dizziness'
  'Panic disorder' --[HAS_SYMPTOM]--> Symptom: 'Sweating'
  'Panic disorder' --[CAN_BE_TREATED_WITH]--> Treatment: 'Relaxation Techniques'
  'Panic disorder' --[CAN_BE_TREATED_WITH]--> Treatment: 'Cognitive Behavioral Therapy'
  'Panic disorder' --[CAN_BE_TREATED_WITH]--> Treatment: 'Antidepressant medications'
  'Vocal cord polyp' --[HAS_SYMPTOM]--> Symptom: 'Vocal Changes'

🎉 Knowledge graph successfully created with:
   • 2491 nodes (392 medical conditions, 947 symptoms, 1128 treatments)
   • 3358 relationships
   • Primary relationship types: HAS_SYMPTOM (1624), CAN_BE_TREATED_WITH (1104)


In [14]:
# Step 5: Create indexes for better performance
print("Creating indexes for better query performance...")

# Create indexes for node properties that we'll frequently search on
indexes = [
    "CREATE INDEX medical_condition_id IF NOT EXISTS FOR (n:`Medical Condition`) ON (n.id)",
    "CREATE INDEX symptom_id IF NOT EXISTS FOR (n:Symptom) ON (n.id)",
    "CREATE INDEX treatment_id IF NOT EXISTS FOR (n:Treatment) ON (n.id)",
    "CREATE INDEX medication_id IF NOT EXISTS FOR (n:Medication) ON (n.id)",
]

for index_query in indexes:
    try:
        graph.query(index_query)
        print(f"✓ Created index: {index_query.split('FOR')[1].split('ON')[0].strip()}")
    except Exception as e:
        print(f"Index already exists or error: {str(e)}")

print("✓ Database indexes created successfully!")

Creating indexes for better query performance...
✓ Created index: (n:`Medical Condition`)
✓ Created index: (n:Symptom)
✓ Created index: (n:Treatment)
✓ Created index: (n:Medication)
✓ Database indexes created successfully!


In [21]:
# Step 6: Set up RAG system for knowledge graph querying
from langchain_neo4j import GraphCypherQAChain

print("Setting up knowledge graph RAG system...")


# Set up GraphCypherQAChain for knowledge graph querying
# Note: Setting allow_dangerous_requests=True for demo purposes
# In production, ensure proper database permissions and security
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    allow_dangerous_requests=True  # Required for security acknowledgment
)

print("✓ Graph QA chain setup complete!")

# Create helper functions for different types of medical queries
def get_symptoms_for_condition(condition_name):
    """Get all symptoms for a specific medical condition"""
    query = f"""
    MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
    WHERE toLower(c.id) CONTAINS toLower('{condition_name}')
    RETURN c.id as condition, collect(s.id) as symptoms
    """
    result = graph.query(query)
    return result

def get_treatments_for_condition(condition_name):
    """Get all treatments for a specific medical condition"""
    query = f"""
    MATCH (c:`Medical Condition`)-[r]->(t:Treatment)
    WHERE toLower(c.id) CONTAINS toLower('{condition_name}')
    RETURN c.id as condition, type(r) as relationship_type, collect(t.id) as treatments
    """
    result = graph.query(query)
    return result

def find_conditions_by_symptoms(symptoms_list):
    """Find medical conditions that have any of the specified symptoms"""
    symptoms_str = "', '".join(symptoms_list)
    query = f"""
    MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
    WHERE toLower(s.id) IN ['{symptoms_str.lower()}']
    RETURN c.id as condition, collect(s.id) as matching_symptoms, count(s) as symptom_count
    ORDER BY symptom_count DESC
    LIMIT 10
    """
    result = graph.query(query)
    return result

print("✓ Helper functions created for medical queries!")

Setting up knowledge graph RAG system...
✓ Graph QA chain setup complete!
✓ Helper functions created for medical queries!


In [22]:
# Step 7: Test the RAG system with sample medical queries
print("🧪 Testing the Medical Knowledge Graph RAG System")
print("=" * 60)

# Test 1: Find symptoms for a specific condition
print("\n1️⃣  Test: What are the symptoms of panic disorder?")
result = get_symptoms_for_condition("panic disorder")
if result:
    condition = result[0]['condition']
    symptoms = result[0]['symptoms']
    print(f"Condition: {condition}")
    print(f"Symptoms: {', '.join(symptoms)}")
else:
    print("No results found")

# Test 2: Find treatments for a condition
print("\n2️⃣  Test: How is panic disorder treated?")
result = get_treatments_for_condition("panic disorder")
if result:
    for row in result:
        condition = row['condition']
        treatments = row['treatments']
        rel_type = row['relationship_type']
        print(f"Condition: {condition}")
        print(f"Treatments ({rel_type}): {', '.join(treatments)}")
else:
    print("No results found")

# Test 3: Find conditions by symptoms
print("\n3️⃣  Test: What conditions might cause chest pain and shortness of breath?")
result = find_conditions_by_symptoms(["chest pain", "shortness of breath"])
if result:
    print("Possible conditions:")
    for row in result:
        condition = row['condition']
        matching_symptoms = row['matching_symptoms']
        count = row['symptom_count']
        print(f"  • {condition} (matches {count} symptoms: {', '.join(matching_symptoms)})")
else:
    print("No matching conditions found")

print("\n" + "=" * 60)

🧪 Testing the Medical Knowledge Graph RAG System

1️⃣  Test: What are the symptoms of panic disorder?
Condition: Panic disorder
Symptoms: Fear of losing control, Trembling, Palpitations, Shortness of breath, Dizziness, Sweating

2️⃣  Test: How is panic disorder treated?
Condition: Panic disorder
Treatments (CAN_BE_TREATED_WITH): Relaxation Techniques, Cognitive Behavioral Therapy, Antidepressant medications

3️⃣  Test: What conditions might cause chest pain and shortness of breath?
Possible conditions:
  • Pulmonary Eosinophilia (matches 2 symptoms: shortness of breath, chest pain)
  • Panic Attack (matches 2 symptoms: shortness of breath, Chest pain)
  • Kidney Failure (matches 2 symptoms: shortness of breath, chest pain)
  • Abscess of the Lung (matches 2 symptoms: shortness of breath, chest pain)
  • Thoracic Aortic Aneurysm (matches 1 symptoms: shortness of breath)
  • Asthma (matches 1 symptoms: shortness of breath)
  • Pulmonary Congestion (matches 1 symptoms: Shortness of breath

In [23]:
print(graph.schema)

Node properties:
Symptom {id: STRING}
Medical Condition {id: STRING}
Treatment {id: STRING}
Medication {id: STRING}
Procedure {id: STRING}
Therapy {id: STRING}
Node {id: STRING}
Therapy Type {id: STRING}
Cause {id: STRING}
Relationship properties:

The relationships:
(:Medical Condition)-[:HAS_SYMPTOM]->(:Symptom)
(:Medical Condition)-[:CAN_BE_TREATED_WITH]->(:Treatment)
(:Medical Condition)-[:CAN_BE_TREATED_WITH]->(:Node)
(:Medical Condition)-[:CAN_BE_TREATED_BY]->(:Treatment)
(:Medical Condition)-[:TREATED_WITH]->(:Treatment)
(:Medical Condition)-[:HAS_TREATMENT]->(:Treatment)
(:Medical Condition)-[:TREATED_BY]->(:Treatment)
(:Medical Condition)-[:HAS_CAUSE]->(:Cause)
(:Treatment)-[:INCLUDES]->(:Medication)
(:Treatment)-[:INCLUDES]->(:Procedure)
(:Treatment)-[:INCLUDES]->(:Treatment)
(:Treatment)-[:INCLUDES]->(:Therapy)
(:Treatment)-[:INCLUDES]->(:Therapy Type)
(:Treatment)-[:INVOLVES_PROCEDURE]->(:Procedure)
(:Treatment)-[:USES_MEDICATION]->(:Medication)


In [42]:
# Advanced Cypher Generation Template with Query Optimization
from langchain import PromptTemplate
ADVANCED_CYPHER_TEMPLATE = """You are an expert Neo4j Cypher query generator for a medical knowledge graph.

DATABASE SCHEMA:
{schema}

MEDICAL KNOWLEDGE GRAPH STRUCTURE:
- Nodes: `Medical Condition`, Symptom, Treatment, Medication, Procedure
- Properties: All nodes have "id" property containing the text value
- Key Relationships:
  * (`Medical Condition`)-[:HAS_SYMPTOM]->(Symptom)
  * (`Medical Condition`)-[:CAN_BE_TREATED_WITH]->(Treatment)
  * (`Medical Condition`)-[:CAN_BE_TREATED_BY]->(Treatment)
  * (`Medical Condition`)-[:TREATED_WITH]->(Treatment)

CRITICAL RULES:
1. ALWAYS use backticks around "Medical Condition" label: `Medical Condition`
2. Use "id" property for all nodes, NOT "name" 
3. Use toLower() and CONTAINS for flexible matching
4. Use collect() to group results
5. Always add LIMIT for multiple results
6. Never use single quotes around node labels - only backticks for labels with spaces

QUERY PATTERNS:

For symptoms questions ("What are symptoms of X", "X symptoms", "signs of X"):
MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE toLower(c.id) CONTAINS toLower("condition_name")
RETURN c.id as condition, collect(s.id) as symptoms
LIMIT 5

For treatment questions ("How to treat X", "treatment for X", "cure for X"):
MATCH (c:`Medical Condition`)-[r:CAN_BE_TREATED_WITH|CAN_BE_TREATED_BY|TREATED_WITH]->(t:Treatment)
WHERE toLower(c.id) CONTAINS toLower("condition_name")
RETURN c.id as condition, collect(DISTINCT t.id) as treatments
LIMIT 5

For diagnostic questions ("What could cause X", "diseases with X symptom", "conditions with X"):
MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE toLower(s.id) CONTAINS toLower("fever") OR toLower(s.id) CONTAINS toLower("high fever")
RETURN c.id as condition, count(s) as symptom_matches
ORDER BY symptom_matches DESC
LIMIT 10

For general condition info ("Tell me about X", "What is X"):
MATCH (c:`Medical Condition`)
WHERE toLower(c.id) CONTAINS toLower("condition_name")
OPTIONAL MATCH (c)-[:HAS_SYMPTOM]->(s:Symptom)
OPTIONAL MATCH (c)-[r:CAN_BE_TREATED_WITH|CAN_BE_TREATED_BY|TREATED_WITH]->(t:Treatment)
RETURN c.id as condition, collect(DISTINCT s.id) as symptoms, collect(DISTINCT t.id) as treatments
LIMIT 5

Question: {question}

Based on the question above, generate the appropriate Cypher query. For the question about "high fever", use this pattern:

MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE toLower(s.id) CONTAINS toLower("fever")
RETURN c.id as condition, count(s) as symptom_matches
ORDER BY symptom_matches DESC
LIMIT 10

Generate ONLY the Cypher query without any explanation:"""

# Create an improved QA prompt template for better responses
QA_TEMPLATE = """You are a helpful medical assistant. Based on the following information from a medical knowledge graph, provide a clear and helpful response to the patient's question.

Context from knowledge graph:
{context}

Question: {question}

Please provide a response as if you are talking to a patient. Be empathetic, clear, and always remind them to consult with healthcare professionals for proper diagnosis and treatment.

Answer:"""

# Create the most optimized chain with both improved templates
optimized_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    cypher_prompt=PromptTemplate(
        input_variables=["schema", "question"], 
        template=ADVANCED_CYPHER_TEMPLATE
    ),
    qa_prompt=PromptTemplate(
        input_variables=["context", "question"],
        template=QA_TEMPLATE
    ),
    return_intermediate_steps=True
)

print("🚀 Improved medical AI assistant chain created!")

# Test with the same question
test_question = "what disease could i have if i am having high fever?"
print(f"\n🧪 Testing improved template: {test_question}")

try:
    result = optimized_chain.invoke({"query": test_question})
    print(f"✓ Answer: {result['result']}")
    if 'intermediate_steps' in result:
        print(f"🔧 Generated Cypher: {result['intermediate_steps'][0]['query']}")
except Exception as e:
    print(f"Error: {str(e)}")

🚀 Improved medical AI assistant chain created!

🧪 Testing improved template: what disease could i have if i am having high fever?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:`Medical Condition`)-[:HAS_SYMPTOM]->(s:Symptom)
WHERE toLower(s.id) CONTAINS toLower("high fever")
RETURN c.id as condition, count(s) as symptom_matches
ORDER BY symptom_matches DESC
LIMIT 10
Full Context:
[{'condition': 'Dengue Fever', 'symptom_matches': 1}, {'condition': 'Influenza (Flu)', 'symptom_matches': 1}]

> Finished chain.
✓ Answer: I'm sorry to hear that you're experiencing a high fever. Based on the information available, high fever can be a symptom of several conditions, including Dengue Fever and Influenza (Flu). However, it's important to remember that many different illnesses can cause a high fever, and only a healthcare professional can provide an accurate diagnosis.

I strongly recommend that you consult with a doctor or visit a healthcare facility as soon as possible.

In [46]:
# Type your medical question when prompted:
my_question = input("Enter your medical question: ")

# Get the answer
result = optimized_chain.invoke({"query": my_question})
print(f"\nQuestion: {my_question}")
print(f"Answer: {result['result']}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:'Medical Condition')
WHERE toLower(c.id) CONTAINS toLower("influenza")
OPTIONAL MATCH (c)-[:HAS_SYMPTOM]->(s:Symptom)
OPTIONAL MATCH (c)-[r:CAN_BE_TREATED_WITH|CAN_BE_TREATED_BY|TREATED_WITH]->(t:Treatment)
RETURN c.id as condition, collect(DISTINCT s.id) as symptoms, collect(DISTINCT t.id) as treatments
LIMIT 5



CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input ''Medical Condition'': expected a node label/relationship type name, '$', '%' or '(' (line 2, column 10 (offset: 16))
"MATCH (c:'Medical Condition')"
          ^}